In [ ]:
from pathlib import Path
import os

DAY = "20251201"
Version = "v1"

PROJECT_ROOT = Path(os.getenv("LLMSC_ROOT", ".")).resolve()

DATA_DIR = Path(os.getenv("LLMSC_DATA_DIR", PROJECT_ROOT / "input")).resolve()
OUT_DIR  = Path(os.getenv("LLMSC_OUT_DIR",  PROJECT_ROOT / "runs" / f"{DAY}.{Version}")).resolve()
OUT_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
import scipy
import scipy.sparse
from sklearn.metrics import roc_auc_score
import adjustText
import gc
import time
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
from sklearn.metrics import confusion_matrix

import logging
logging.basicConfig(
    level=logging.INFO,
    format="%(levelname)s:%(name)s:%(message)s"
)

from llm_sc_curator import LLMscCurator
from llm_sc_curator.masking import FeatureDistiller


import warnings
warnings.filterwarnings("ignore")

In [ ]:
RANDOM_SEED = 42
import random, os
import numpy as np
random.seed(RANDOM_SEED)
os.environ['PYTHONHASHSEED'] = str(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

print(f"🔒 Random seed set to {RANDOM_SEED} for reproducibility.")

🔒 Random seed set to 42 for reproducibility.


In [ ]:
def marker_effects_csv(
    adata,
    label_col: str,
    genes: list[str],
    out_csv: str,
    *,
    layer_for_fc: str | None = None,   # counts があれば "counts"
    expr_threshold: float = 0.0,
    pseudocount: float = 1e-3,
):
    labels = adata.obs[label_col].astype(str).unique().tolist()

    # gene present only
    genes = [g for g in genes if g in adata.var_names]

    def _vec(g, layer=None):
        X = adata[:, g].layers[layer] if (layer is not None and layer in adata.layers) else adata[:, g].X
        if hasattr(X, "toarray"):
            X = X.toarray()
        return np.asarray(X).reshape(-1)

    rows = []
    for lab in labels:
        y = (adata.obs[label_col].astype(str).values == lab)

        for g in genes:
            x = _vec(g, layer=None)  # for AUC / detection
            # detection
            det_t = float((x[y] > expr_threshold).mean())
            det_o = float((x[~y] > expr_threshold).mean())

            # AUROC
            auc = np.nan
            if np.nanstd(x) > 0 and y.sum() > 0 and (~y).sum() > 0:
                auc = float(roc_auc_score(y.astype(int), x))

            # log2FC (prefer counts if available)
            x_fc = _vec(g, layer=layer_for_fc) if (layer_for_fc and layer_for_fc in adata.layers) else x
            mu_t = float(np.mean(x_fc[y]))
            mu_o = float(np.mean(x_fc[~y]))
            log2fc = float(np.log2((mu_t + pseudocount) / (mu_o + pseudocount)))

            rows.append({
                "label": lab,
                "gene": g,
                "log2FC": log2fc,
                "delta_det": det_t - det_o,
                "AUROC": auc,
            })

    df = pd.DataFrame(rows)
    df.to_csv(out_csv, index=False)
    return df


In [ ]:
file_path = "/runs/20251201.v1/OSCC_visium_llmsc_finetype.h5ad"

adata_vis = sc.read_h5ad(file_path)

print("--- Loaded AnnData Info ---")
print(adata_vis)

print("\n--- obs columns ---")
print(adata_vis.obs[["LLMsc_fine"]].head())

--- Loaded AnnData Info ---
AnnData object with n_obs × n_vars = 983 × 4000
    obs: 'in_tissue', 'array_row', 'array_col', 'imagecol', 'imagerow', 'pathology', 'category', 'cluster', 'sample', 'n_counts', 'batch', '_scvi_batch', '_scvi_labels', 'leiden', 'sample_density', '_scvi_raw_norm_scaling', 'Niche', 'leiden_coarse', 'major_type', 'fine_type', 'LLMsc_fine'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells-0-0', 'n_cells-1-0', 'n_cells-1', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm', 'highly_variable_nbatches'
    uns: 'Niche_colors', '_scvi', 'dendrogram_leiden', 'hvg', 'leiden', 'leiden_coarse', 'leiden_colors', 'llm_reasoning', 'neighbors', 'pathology_colors', 'pca', 'sample_colors', 'sample_density_params', 'spatial', 'umap', 'wilcoxon'
    obsm: 'X_pca', 'X_scvi', 'X_umap', 'spatial'
    varm: 'PCs'
    layers: 'counts', 'scvi_expr'
    obsp: 'connectivities', 'distances'

--- obs columns ---
                                      

In [ ]:
# ==========================================
# 1. Standardize label wording
# ==========================================

label_map = {
    # --- Tumor / Malignant ---
    "Muscle-Infiltrating Carcinoma (Hypoxic, ISG-high)": "Tumor (Invasive/Hypoxic)",
    "Myofibroblastic Carcinoma":                         "Tumor (Myofibroblastic/EMT)",
    "Malignant Keratinocyte (Partial EMT, IFN-response, proliferating)": "Tumor (p-EMT/proliferating/ISG)",
    "Malignant Keratinocyte (Partial EMT)":              "Tumor (p-EMT)",
    "Basal-like carcinoma cell (ISG-high)":              "Tumor (Basal/ISG)",
    "Basal-like carcinoma cell (EMT, proliferating)":    "Tumor (Basal/proliferating)",
    "Partial EMT carcinoma cell (Hypoxic)":              "Tumor (p-EMT/Hypoxic)",
    "Hypoxic-Mesenchymal Carcinoma cell":                "Tumor (Mesenchymal/Hypoxic)",

    # --- Stromal / CAF  ---
    "Myofibroblastic Cancer-Associated Fibroblast":      "CAF (Myofibroblastic)", # or myCAF
    "Matrix-remodeling Cancer-Associated Fibroblast (CAF) Niche": "CAF (Matrix-remodeling)", # or mCAF

    # --- Structures ---
    "Tertiary Lymphoid Structure (TLS)":                 "TLS (Tertiary Lymphoid Structure)"
}

In [ ]:
def harmonize_labels(adata, col="fine_type", mapping=None, new_col=None):
    """
    Apply a user-provided mapping dict to harmonize LLM labels.

    This is intentionally a thin wrapper: all ontology decisions are
    encoded explicitly in `mapping` and stay outside the core engine.
    """
    if mapping is None:
        return adata

    if new_col is None:
        new_col = f"{col}_harmonized"

    adata.obs[f"{col}_raw"] = adata.obs[col].astype("string")
    adata.obs[new_col] = adata.obs[col].replace(mapping).astype("category")
    return adata

In [ ]:
fine_col = "LLM_fine_label"

adata_vis = harmonize_labels(
    adata_vis,
    col="LLMsc_fine",
    mapping=label_map,
    new_col=fine_col
)

fine_order = [
    # --- Tumor / Malignant (Tumor) ---
    "Tumor (p-EMT/Hypoxic)",
    "Tumor (Basal/ISG)",
    "Tumor (Basal/proliferating)",
    "Tumor (Mesenchymal/Hypoxic)",
    "Tumor (Invasive/Hypoxic)",
    "Tumor (Myofibroblastic/EMT)",
    "Tumor (p-EMT/proliferating/ISG)",
    "Tumor (p-EMT)",

    # --- Stromal / CAF (CAF) ---
    "CAF (Myofibroblastic)", # or myCAF
    "CAF (Matrix-remodeling)", # or mCAF

    # --- Structures ---
    "TLS (Tertiary Lymphoid Structure)"
]

present = [c for c in fine_order if c in adata_vis.obs[fine_col].cat.categories]
adata_vis.obs[fine_col] = pd.Categorical(
    adata_vis.obs[fine_col],
    categories=present,
    ordered=True,
)

print(adata_vis.obs[fine_col].value_counts())

LLM_fine_label
Tumor (Invasive/Hypoxic)             186
TLS (Tertiary Lymphoid Structure)    139
Tumor (Myofibroblastic/EMT)          129
CAF (Myofibroblastic)                108
Tumor (p-EMT/proliferating/ISG)      105
CAF (Matrix-remodeling)               86
Tumor (p-EMT)                         64
Tumor (Basal/ISG)                     59
Tumor (Basal/proliferating)           51
Tumor (p-EMT/Hypoxic)                 30
Tumor (Mesenchymal/Hypoxic)           26
Name: count, dtype: int64


In [ ]:
oscc_markers = [
    # --- Epithelial / Basal ---
    "KRT5", "KRT14", "TP63", "EPCAM",
    # --- Proliferation ---
    "MKI67", "TOP2A",
    # --- Interferon / ISG ---
    "ISG15", "IFIT1", "MX1", "STAT1",
    # --- Hypoxia ---
    "HIF1A", "VEGFA", "CA9", "SLC2A1",
    # --- Partial EMT ---
    "LAMC2", "TNC", "PDPN", "ITGA5",
    # --- Mesenchymal / Myofibroblastic ---
    "VIM", "FN1", "MMP9", "ACTA2", "COL1A1", "TAGLN",
    # Immune / TLS
    "CD3D", "MS4A1", "PTPRC"
]
# Focus only on genes present in the dataset.
valid_markers = [g for g in oscc_markers if g in adata_vis.var_names]

In [ ]:
# Visium（OSCC）
label_col = "LLM_fine_label"
genes = valid_markers
layer_for_fc = "counts" if "counts" in adata_vis.layers else None

df_vis = marker_effects_csv(
    adata_vis,
    label_col=label_col,
    genes=genes,
    out_csv=f"{outdir}/EDFig3c_marker_effects.csv",
    layer_for_fc=layer_for_fc,
)
df_vis.sort_values(["AUROC","log2FC"], ascending=[False, False]).head(20)

,label,gene,log2FC,delta_det,AUROC
43,Tumor (p-EMT/Hypoxic),KRT14,2.942547,0.000000,0.995873
42,Tumor (p-EMT/Hypoxic),KRT5,2.491465,0.000000,0.979468
0,Tumor (Basal/ISG),KRT5,2.132186,0.000000,0.953977
1,Tumor (Basal/ISG),KRT14,2.223759,0.000000,0.943429
44,Tumor (p-EMT/Hypoxic),TP63,2.581514,0.089192,0.929941
55,Tumor (p-EMT/Hypoxic),PDPN,2.422189,0.039874,0.921056
53,Tumor (p-EMT/Hypoxic),LAMC2,2.123770,0.004197,0.869010
52,Tumor (p-EMT/Hypoxic),SLC2A1,2.972583,0.379853,0.867716
37,Tumor (Mesenchymal/Hypoxic),FN1,1.137573,0.000000,0.857246
49,Tumor (p-EMT/Hypoxic),HIF1A,2.425052,0.122770,0.852239


In [ ]:
file_path = "/runs/20251201.v1/Xenium_L1_vs_LLMsc_finetype.h5ad"

adata_xen = sc.read_h5ad(file_path)

print("--- Loaded AnnData Info ---")
print(adata_xen)

print("\n--- obs columns ---")
print(adata_xen.obs[["fine_type"]].head())

--- Loaded AnnData Info ---
AnnData object with n_obs × n_vars = 88863 × 422
    obs: 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'deprecated_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'sample_id', 'sum', 'detected', 'altexps_NegControlProbe_sum', 'altexps_NegControlProbe_detected', 'altexps_NegControlProbe_percent', 'altexps_UnassignedCodeword_sum', 'altexps_UnassignedCodeword_detected', 'altexps_UnassignedCodeword_percent', 'altexps_NegControlCodeword_sum', 'altexps_NegControlCodeword_detected', 'altexps_NegControlCodeword_percent', 'total', 'ol', 'PIGR', 'IGHG3', 'CEACAM6', 'Leiden', 'Level2', 'Level1', 'Level0', 'x_centroid', 'y_centroid', 'leiden_coarse', 'major_type', 'fine_type', 'x', 'y'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'Level1_colors', 'fine_type_colors', 'hvg', 'leiden_coarse', 'llm_reasoning', 'neighbors', 'pca'
    obsm: 'X_pca', 'spatial'
    varm: '

In [ ]:
fine_col = "LLM_fine_label"

label_map =  {
        "Absorptive Colonocyte": "Absorptive colonocyte",
    }

adata_xen = harmonize_labels(
    adata_xen,
    col="fine_type",
    mapping=label_map,
    new_col=fine_col
)

fine_order = [
  # --- Epithelial Niche (Crypt base -> Top) ---
  "Colorectal cancer stem cell",
  "Regenerative Colorectal Cancer Stem Cell",
  "Colorectal cancer stem cell (proliferating)",
  "Deep Crypt Secretory cell",
  "Enteroendocrine progenitor cell",
  "Immature Goblet cell",
  "Absorptive Goblet Cell",
  "REG4+ Goblet cell",
  "Absorptive colonocyte",

  # --- Stromal ---
  "Pericyte-like CAF",
  "Myofibroblastic Cancer-Associated Fibroblast (mCAF)",
  "Antigen-presenting Cancer-Associated Fibroblast (apCAF)",
  "Endothelial cell",

  # --- Immune ---
  "Mast cell",
  "FCN1+ Inflammatory Macrophage",
  "Antigen-presenting Tumor-Associated Macrophage",
  "SPP1+ Tumor-Associated Macrophage",
  "SPP1+ Tumor-Associated Macrophage (ISG-high)",
  "T cell",
  "CD8+ Cytotoxic T cell (activated)",

  # --- B / Plasma ---
  "Plasmablast",
  "Plasma cell",
  "Plasma cell (proliferating)",
  "Plasma cell (PE]CAM1+)",
  "IgA plasma cell",
  "IgG3+ Plasma cell"
]

present = [c for c in fine_order if c in adata_xen.obs[fine_col].cat.categories]
adata_xen.obs[fine_col] = pd.Categorical(
    adata_xen.obs[fine_col],
    categories=present,
    ordered=True,
)

print(adata_xen.obs[fine_col].value_counts())

LLM_fine_label
Colorectal cancer stem cell (proliferating)                10910
Colorectal cancer stem cell                                 9197
Absorptive colonocyte                                       7898
Regenerative Colorectal Cancer Stem Cell                    7462
Antigen-presenting Tumor-Associated Macrophage              6966
T cell                                                      5351
Myofibroblastic Cancer-Associated Fibroblast (mCAF)         4187
Endothelial cell                                            3963
Mast cell                                                   3804
Antigen-presenting Cancer-Associated Fibroblast (apCAF)     3461
Pericyte-like CAF                                           3441
Immature Goblet cell                                        2807
Deep Crypt Secretory cell                                   2454
IgA plasma cell                                             2430
SPP1+ Tumor-Associated Macrophage (ISG-high)                2132
IgG3+ Plas

In [ ]:
canonical_genes = [
  # Stem / Cycling
  "LGR5", "OLFM4", "ASCL2", "AXIN2", "SMOC2", "MKI67", "TOP2A",
  # Secretory / Goblet
  "MUC2", "SPINK4", "REG4", "ATOH1", "NEUROD1", "CHGA",
  # Absorptive
  "KRT20", "FABP1", "CA1", "AQP8",
  # Stromal / Endo
  "COL1A1", "FAP", "ACTA2", "RGS5", "GREM1", "PECAM1", "PLVAP",
  # Myeloid
  "CD68", "SPP1", "C1QA", "FCN1", "S100A8", "TPSAB1",
  # T / NK
  "CD3E", "CD8A", "FOXP3", "GZMB", "CCL4", "IL7R",
  # B / Plasma
  "CD79A", "MS4A1", "JCHAIN", "MZB1", "IGHA1", "IGHG3"
]

In [ ]:
# Colon cancer Xenium
label_col = "fine_type"
genes = canonical_genes
layer_for_fc = "counts" if "counts" in adata_xen.layers else None

df_xe = marker_effects_csv(
    adata_xen,
    label_col=label_col,
    genes=genes,
    out_csv=f"{outdir}/EDFig3d_marker_effects.csv",
    layer_for_fc=layer_for_fc,
)
df_xe.sort_values(["AUROC","log2FC"], ascending=[False, False]).head(20)



,label,gene,log2FC,delta_det,AUROC
159,Plasma cell (PECAM1+),IGHG3,4.370898,0.751134,0.998838
756,SPP1+ Tumor-Associated Macrophage,SPP1,6.592311,0.951913,0.994845
798,Plasmablast,MZB1,4.151627,0.850641,0.991784
720,Myofibroblastic Cancer-Associated Fibroblast (...,GREM1,5.227987,0.902499,0.987039
609,Deep Crypt Secretory cell,OLFM4,4.006798,0.864320,0.977057
545,Immature Goblet cell,OLFM4,4.021277,0.868606,0.976125
350,Plasma cell,MZB1,3.759775,0.851954,0.975345
191,IgG3+ Plasma cell,IGHG3,3.500660,0.751186,0.974855
582,REG4+ Goblet cell,REG4,4.168370,0.838431,0.962942
458,Absorptive Colonocyte,FABP1,2.902914,0.620732,0.959762
